# Setup


In [ ]:
setwd("/project/kohl_analysis/analysis/CAdir/results/results/")
renv::load("/project/kohl_analysis/analysis/CAdir")
devtools::load_all("/home/kohl/PhD/gits/ClemensKohl/CAdir")
source("./utils.R")

suppressPackageStartupMessages({
  library(APL)
  library(SingleCellExperiment)
  library(dplyr)
  library(tidyr)

  # To load the data set
  library(TENxPBMCData)
  library(Seurat)
  library(SeuratObject)
  library(scater)
  library(scuttle)
  library(scran)

  # Plotting
  library(ggsankey)
  # devtools::install_github("teunbrand/gguidance")
  library(gguidance)
})

dir <- "/project/kohl_analysis/analysis/CAdir/results/results/"
imgdir <- file.path(dir, "img/figure2/")
dir.create(imgdir, recursive = TRUE)

options(repr.plot.width = 20, repr.plot.height = 15)

## Load data


In [ ]:
sce <- sce_pbmc3k()

# CAdir


In [ ]:
set.seed(1234)

sce_bu <- sce
sce.dec <- scran::modelGeneVar(sce)
sce.top <- scran::getTopHVGs(sce.dec, prop = 0.2, var.threshold = NULL)
sce <- sce[sce.top, ]
sce <- runUMAP(sce, ntop = 2000)

ca <- cacomp(
    obj = as.matrix(logcounts(sce)),
    princ_coords = 3,
    dims = 30,
    top = nrow(sce),
    residuals = "pearson",
    python = TRUE,
    clip = TRUE
)

With `cutoff = NULL` CAdir tries to estimate the angle cutoff directly from the data.

## Split & Merge Clustering

In [ ]:
set.seed(2358)
cak <- dirclust_splitmerge(
    caobj = ca,
    k = 15,
    cutoff = NULL,
    method = "random",
    apl_quant = 0.99,
    counts = NULL,
    min_cells = 30,
    reps = 5,
    make_plots = TRUE,
    apl_cutoff_reps = 100,
    qcutoff = 0.6 # 0.1
)

cak <- rank_genes(cadir = cak, caobj = ca)
top <- top_genes(cak)

### Annotate clusters

In [ ]:
cak <- annotate_biclustering(
    obj = cak,
    universe = rownames(sce),
    org = "hs"
)

cak <- rank_genes(cadir = cak, caobj = ca)
cak

sce$cadir <- cak@cell_clusters

um1 <- plotUMAP(sce, colour = "cadir")
um2 <- plotUMAP(sce, colour = "cell_type")

ari <- aricode::clustComp(sce$cadir, sce$cell_type)
p <- um1 + ggtitle(paste0("ARI: ", round(ari$ARI, 2))) + um2
p

ggsave(
  plot = p,
  file = file.path(imgdir, "umap.png"),
  width = 3600,
  height = 1800,
  units = "px"
)

## Association plots of clusters

In [ ]:
pc <- plot_clusters(
  cak,
  ca,
  show_genes=T,
  label_genes = T,
  ntop = 5
)

ggsave(
  plot = pc,
  file = file.path(imgdir, "plot_clusters.pdf"),
  width = 3400,
  height = 2000,
  units ="px"
)

pc_mod <- plot_clusters_custom(
  cak,
  ca,
  show_genes=T,
  label_genes = T,
  ntop = 5,
  outlier_cluster = "cluster_5"
)

pc_mod
ggsave(
  plot = pc_mod,
  file = file.path(imgdir, "plot_clusters.pdf"),
  width = 3400,
  height = 2000,
  units ="px"
)

### Identify outlier genes & cells

In [ ]:
cak <- rank_genes(cak, ca)
rnks <- cak@gene_ranks$cluster_5
p_expr <- scater::plotExpression(sce, features = head(rnks$Rowname, 4), x = "cadir", colour_by = "cadir", ncol = 2) +
  scale_color_mpimg() +
  labs(x = "Cluster", title = "Top 4 genes cluster 5") +
  theme_bw(base_size = 16) +
  theme(
  axis.text.x = element_text(angle = 45, hjust = 1, color = "black"),
  legend.position = "none"
 )

p_expr

ggsave(
  plot = p_expr,
  file = file.path(imgdir, "cluster_5_top_genes.pdf"),
  width = 2200,
  height = 2000,
  units = "px"
)

#### Correct outliers

In [ ]:
cell_rnk <- get_top_cells(cadir = cak, caobj = ca, cluster = "cluster_5")
top_cells <- cell_rnk$Colname[cell_rnk$Score >= 0]
head(cell_rnk)

Cells that are strongly associated to the cluster:

In [ ]:
p_cl5_cells <- cluster_apl(
  ca,
  cak,
  cluster = "cluster_5",
  direction = cak@directions["cluster_5", ],
  group = which(cak@cell_clusters == "cluster_5"),
  highlight_cluster = TRUE,
  show_genes = TRUE,
  label_genes = FALSE,
  show_lines = FALSE,
  size_factor = 0.5,
  ntop = 5
  ) +
ggrepel::geom_label_repel(
    data = cell_rnk[cell_rnk$Score >= 0,],
  ggplot2::aes(
    x = x,
    y = y,
    label = Colname
  ),
  color = "#c6d325",
  box.padding = 1,
  max.overlaps = Inf
)

p_cl5_cells
ggsave(
  plot = p_cl5_cells,
  file = file.path(imgdir, "cluster_5_cells.pdf"),
  width = 1600,
  height = 1600,
  units = "px"
)

Remove outliers:

In [ ]:
sce_corr <- sce[(!rownames(sce) %in% rnks$Rowname[rnks$Score >= 0]), (!colnames(sce) %in% top_cells)]

ca_corr <- cacomp(
    obj = as.matrix(logcounts(sce_corr)),
    princ_coords = 3,
    dims = 30,
    top = nrow(sce_corr),
    residuals = "pearson",
    python = TRUE,
    clip = TRUE
)

set.seed(2358)
cak_corr <- dirclust_splitmerge(
    caobj = ca_corr,
    k = 15,
    cutoff = NULL,
    method = "random",
    apl_quant = 0.99,
    counts = NULL,
    min_cells = 30,
    reps = 5,
    make_plots = TRUE,
    apl_cutoff_reps = 100,
    qcutoff = 0.6
)
# cak_corr@gene_clusters <- assign_genes(cadir = cak_corr, caobj = ca_corr, qcutoff = 0.8)
cak_corr <- annotate_biclustering(
    obj = cak_corr,
    universe = rownames(sce_corr),
    org = "hs"
)

cak_corr <- rank_genes(cadir = cak_corr, caobj = ca_corr)

pc_corr <- plot_clusters(
  cak_corr,
  ca_corr,
  show_genes = T,
  label_genes = T,
  # ntop = 5
)

cak_corr
pc_corr

ggsave(
  plot = pc_corr,
  file = file.path(imgdir, "apl_clusters_corrected.pdf"),
  width = 3400,
  height = 2000,
  units = "px"
)

Sankey plot showing to which clusters the remaining cells from cluster 5 are being clustered.

In [ ]:
sce_corr$cadir2 <- cak_corr@cell_clusters
sce_sank <- sce_corr[,sce_corr$cadir == "cluster_5"]

data <- colData(sce_sank) %>%
        as.data.frame() %>%
        rename('First' = "cadir",
               'Second' = "cadir2")

df <- data %>%
    make_long(First, Second)

info  <-  data.frame(Biclusters = as.factor(data$First),
                     Truth = as.character(data$Second)) %>%
                as_tibble() %>%
                group_by(Biclusters, Truth) %>%
                summarise(n = n()) %>%
                group_by(Truth) %>%
                mutate(Partition = n/sum(n)) %>%
                ungroup() %>%
                as.data.frame() %>%
                group_by(Truth) %>%
                filter(Partition == max(Partition) ) %>%
                dplyr::select(Biclusters, Truth) %>%
                distinct(Biclusters, .keep_all = TRUE)


pal <- mpimg_pal()

cols <- pal(length(unique(sce_sank$cadir)))
names(cols) <- unique(sce_sank$cadir)

info <- left_join(data.frame("Biclusters" = names(cols), "col" = cols), info, by = "Biclusters")
info$col <- pal(nrow(info) + 1)[2:3]

cols <- c(cols, info$col)

names(cols) <- c(as.character(unique(sce_sank$cadir)), info$Truth)


order <- c(
  "cluster_5",
  "Naive_CD4+_T_cell",
  "Natural_killer_cell"
)


wrap <- 17
df$node <- factor(stringr::str_wrap(df$node, wrap), levels = stringr::str_wrap(order, wrap))
df$next_node <- factor(stringr::str_wrap(df$next_node, wrap), levels = stringr::str_wrap(order, wrap))

names(cols) <- stringr::str_wrap(names(cols), wrap)


sank <- ggplot(df, aes(x = x,
               next_x = next_x,
               node = node,
               next_node = next_node,
               fill = node,
               label = as.character(node))) +
  geom_sankey(flow.alpha = 0.6, flow.fill = c("#777777") ) +
  geom_sankey_label(size = 6) +
  labs(x = "") +
  scale_fill_manual(values = cols) +
  theme_sankey(base_size = 26) +
  theme(
    legend.position = "none"
  )

sank

ggsave(plot = sank,
       filename = file.path(imgdir, "sankey_plot_cluster_5.pdf"),
       width = 2700,
       height = 2000,
       units = "px")

### Platelet cells


In [ ]:
cak_corr_mod <- cak_corr

goa_res <- CAbiNet::per_cluster_goa(
  cabic = cak_corr_mod,
  universe = rownames(sce_corr),
  set = "CellMarker",
  org = "hs",
  min_size = 10,
  max_size = 500
)
head(goa_res$Megakaryocyte)

Top 5 most highly significant gene sets for cluster "Megakaryocyte":

In [ ]:
cl_goa <- goa_res$Megakaryocyte

ggplot(cl_goa[1:5, ], aes(x = gene_set, y = -log10(padj), fill = -log10(padj))) +
  geom_col() +
  scale_fill_gradientn(
    colours = c("#29485d", "#006c66", "#c6d325", "#ef7c00"),
  ) + 
  theme_bw()

Gene Set enrichment over all corrected clusters:

In [ ]:
nclust <- length(goa_res)

goa_mat <- lapply(goa_res, function(x, nc = nclust) {
  subs <- min(nrow(x), nc)
  x[seq_len(subs), ]
})

goa_mat <- dplyr::bind_rows(goa_mat, .id = "cluster")

gr <- goa_mat[,c("cluster", "gene_set", "padj")]
gr <- complete(gr, cluster, gene_set, fill = list(padj = 1))

p_gs_enr <- ggplot(gr, aes(x = cluster, y = gene_set, fill = -log10(padj))) +
  geom_tile() +
  scale_fill_gradientn(
    colours = c("#29485d", "#006c66", "#c6d325", "#ef7c00"),
    limits = c(0, quantile(-log10(gr$padj), 0.99)),
    oob = scales::squish,
    guide = "colourbar_custom"
  ) + 
  theme_bw() +
  labs(x = "Cluster", y = "Gene Set", title = "Gene Set enrichment") +
  theme(
  axis.text.x = element_text(angle = 45, hjust = 1, color = "black"),
  axis.text.y = element_text(color = "black"),
    text = element_text(size = 16),
    plot.title = element_text(size = 16, face = "plain"))  #"bold"

p_gs_enr

ggsave(
  plot = p_gs_enr,
  file = file.path(imgdir, "gene_set_enr_hm.pdf"),
  width = 2000,
  height = 3000,
  units = "px"
)

## Split & Merge plot

In [ ]:
sm <- sm_plot(
  cadir = cak,
  caobj = ca,
  rm_redund = TRUE,
  keep_end = TRUE,
  highlight_cluster = TRUE,
  show_genes = F,
  annotate_clusters = TRUE,
  org = "hs"
)
sm

ggsave(plot = sm,
  file = file.path(imgdir, "split_merge_plot.pdf"),
  width = 3000,
  height = 1800,
  units = "px")

In [ ]:
sessionInfo()